In [1]:
import os
import argparse
from ta_pet_id.core.utils import get_package_path
from ta_pet_id.core.base_utils import silence_all_warnings

In [3]:
import pandas as pd
from ta_pet_id.core.context import create_context as _create_context
from ta_pet_id.pipeline.main import (enroll_pet as _enroll_pet,
                                     infer_pet_id as _infer_pet_id,
                                     train_efficientnet as _train_efficientnet,
                                     train_yolo as _train_yolo)
from ta_pet_id.data_prep import db_utils
from ta_pet_id.data_prep import core
from ta_pet_id.pipeline import yolo, efficientnet
from ta_pet_id.evaluation.report import get_pipeline_eval_report as _get_pipeline_eval_report

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
exceptioncannot open resource
exceptioncannot open resource
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
# !pip install opencv-python
# !pip install torch
!pip install torchvision opencv-python torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 64.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 86.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.7 MB/s eta 0:00:0000:0100:01
     ━━━━━

In [4]:
def run_enrollment(conf_path, re_enroll=False):
    """Run the enrollment process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder
    re_enroll : bool
      whether to re-enroll the existing enrolled pets or not
      Note: To overwrite existing enrollment, set it to True

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # get list of existing enrolled pets
    enrolled_pets = []
    append_new_pets = False
    if not re_enroll:
        enrolled_pets = db_utils.load_enrolled_pet_db(context_obj)['house_pet_id'].unique()
        append_new_pets = True

    # prepare enroll metadata from the raw image data
    enroll_meta_df = core.prep_enroll_data(context_obj, enrolled_pets)
    if len(enroll_meta_df) == 0:
        raise Warning(f"All the pets from the given enroll data are already enrolled! If you want to re-enroll, "
                      f"run the process again with re_enroll=True.")
    # save enroll metadata to the DB
    db_utils.save_enroll_metadata_db(context_obj, enroll_meta_df, append=append_new_pets)

    # load the trained pipeline models
    eff_net_model = efficientnet.load_model(context_obj)
    yolo_model_objs = yolo.load_model(context_obj)

    # execute enroll process for each house
    house_dfs = []
    for house_id in enroll_meta_df['house_id'].unique():
        house_df = enroll_meta_df[enroll_meta_df['house_id'] == house_id]
        house_pet_db = _enroll_pet(context_obj, house_id, house_df, eff_net_model, yolo_model_objs)
        house_dfs.append(house_pet_db)

    # consolidate households pets into one and save the enrolled pet info to DB
    enrolled_pet_df = pd.concat(house_dfs, ignore_index=True)
    db_utils.save_enrolled_pet_db(context_obj, enrolled_pet_df, append=append_new_pets)
    print("Pet enrollment completed for all houses!")


def run_inference(conf_path, return_eval_report=False):
    """Run the inference process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder
    return_eval_report : bool
      whether to get classification evaluation report or not
      Note: Subject to if truth label instances are provided or not

    Returns
    -------
    report: dict or None
        a report dict if truth labels are provided otherwise None

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # prepare inference metadata from the raw image data
    infer_meta_df = core.prep_inference_data(context_obj)

    # save inference metadata to the DB
    db_utils.save_inference_metadata_db(context_obj, infer_meta_df)

    # load the trained pipeline models
    eff_net_model = efficientnet.load_model(context_obj)
    yolo_model_objs = yolo.load_model(context_obj)

    # load enrolled pets db
    enrolled_pets_db = db_utils.load_enrolled_pet_db(context_obj)
    enrolled_houses = enrolled_pets_db['house_id'].unique()

    # execute inference process for each house
    house_dfs = []
    for house_id in infer_meta_df['house_id'].unique():
        if house_id not in enrolled_houses:
            # as no pets are enrolled from this house, inference can't be given for the same
            continue
        house_df = infer_meta_df[infer_meta_df['house_id'] == house_id]
        # print(house_df)
        house_df_preds = _infer_pet_id(context_obj, house_id, house_df, enrolled_pets_db, eff_net_model,
                                       yolo_model_objs)
        house_dfs.append(house_df_preds)

    # consolidate households pets into one and save the inference data predictions to DB
    inference_preds_df = pd.concat(house_dfs, ignore_index=True)
    db_utils.save_inference_prediction_db(context_obj, inference_preds_df)

    # select the inference data for which truth label are given and generate evaluation report
    eval_df = inference_preds_df[inference_preds_df['pet_type'].notnull() &
                                 inference_preds_df['pet_id'].notnull()]
    if len(eval_df) > 0:
        report_path = context_obj.data_catalog['inference']['inference_preds_report'].format(
            **context_obj.data_catalog['inference'])
        report = _get_pipeline_eval_report(eval_df, return_report=return_eval_report, write_report=True,
                                           report_path=report_path)
    if return_eval_report:
        return report


def train_efficientnet_model(conf_path):
    """Run the EfficientNetB2 training process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # prepare train meta data from the raw image data
    train_meta_df = core.prep_efficientnet_train_metadata(context_obj)
    db_utils.save_efficientnet_train_metadata_db(context_obj, train_meta_df)

    # run efficientnet training process
    _train_efficientnet(context_obj, train_meta_df)


def train_yolo_model(conf_path):
    """Run the YOLOv5 training process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # prepare train meta data from the raw image data
    train_meta_df = core.prep_yolo_train_metadata(context_obj)
    db_utils.save_yolo_train_metadata_db(context_obj, train_meta_df)

    # run yolo training process
    _train_yolo(context_obj, train_meta_df)

In [5]:
config_path="\\home\\ec2-user\\SageMaker\\efficient_pipeline\\cat_visual_id_master_lr\\src\\../production/conf/config.yml"
config_path="../production/conf/config.yml"

In [ ]:
run_enrollment(config_path, re_enroll=True)

Initialized Random Seed : 0
Context created from ../production/conf/config.yml
Package version : 1.0.0
100%|██████████| 1685/1685 [05:34<00:00,  5.04it/s]
Importing a function (__inference_block1a_activation_layer_call_and_return_conditional_losses_2077924) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_block6d_activation_layer_call_and_return_conditional_losses_2082068) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_block1a_se_reduce_layer_call_and_return_conditional_losses_2102657) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_block2c_se_reduce_layer_call_and_return_conditional_losses_2104860) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
Importing a function (__inference_block4b_expand_activation_layer_call_and_return_conditional_l

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


No training configuration found in save file, so the model was *not* compiled. Compile it manually.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1/1 [==============================] - 0s 95ms/step


In [ ]:
run_inference(config_path)

In [9]:
import pandas as pd
df=pd.read_excel("C:/Users/uday.bijjala/cat_visual_id_master_cosine_similarity/data/processed/inference_data/inference_preds_db.xlsx")

In [22]:
df.head()

,datetime,house_id,image_path,pet_type,pet_id,pred_pet_type,pred_pet_id,orig
0,09-08-2023 18:28:40,house_10,house_10\cashew_0.jpg,NaN,NaN,cat,cashew,cashew
1,09-08-2023 18:28:40,house_10,house_10\cashew_1.jpg,NaN,NaN,cat,cashew,cashew
2,09-08-2023 18:28:40,house_10,house_10\liana_0.jpg,NaN,NaN,cat,liana,liana
3,09-08-2023 18:28:40,house_10,house_10\liana_1.jpg,NaN,NaN,cat,liana,liana
4,09-08-2023 18:28:40,house_11,house_11\ferris_1.jpg.jpg,NaN,NaN,cat,ferris,ferris


In [18]:
df["pred_pet_id"]=df["pred_pet_id"].apply(lambda x:x.lower())

In [20]:
def ip(x):
    if x[1] in x[0]:
        return x[1]
    else: 
        j=x[0].split("\\")[1].split(".")[0].split("_")[0]
        return j

df["orig"]=df[["image_path","pred_pet_id"]].apply(ip,axis=1)

In [23]:
from sklearn.metrics import classification_report

print(classification_report(df["orig"],df["pred_pet_id"]))

              precision    recall  f1-score   support

    12562173       1.00      1.00      1.00         2
    12865110       1.00      1.00      1.00         3
    13010040       1.00      1.00      1.00         2
    13525829       1.00      1.00      1.00         2
    13682173       1.00      1.00      1.00         1
    15000244       1.00      1.00      1.00         2
    15437038       1.00      1.00      1.00         3
    16622423       1.00      1.00      1.00         2
    16807081       1.00      1.00      1.00         2
      cashew       1.00      1.00      1.00         2
      ferris       0.50      1.00      0.67         1
        gene       1.00      1.00      1.00         1
         gus       0.00      0.00      0.00         1
        kobe       1.00      1.00      1.00         1
       liana       1.00      1.00      1.00         2

    accuracy                           0.96        27
   macro avg       0.90      0.93      0.91        27
weighted avg       0.94   

C:\Users\uday.bijjala\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\uday.bijjala\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\uday.bijjala\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [76]:
def ip(x):
    if "flop" in x:
        return 0
    if "gremline" in x:
        return 1
df["image_path1"]=df["image_path"].apply(ip)

In [63]:
def ip(x):
    if "jessie" in x:
        return 0
    if "kitty" in x:
        return 1
    if "lilo" in x:
        return 2
    if "shadow" in x: 
        return 3
    if "iris" in x:
        return 4 
    if "new1" in x: 
        return 5
df["image_path1"]=df["image_path"].apply(ip)

In [77]:
def ip(x):
    if "flop" in x[1]:
        return 0
    if "gremline" in x[1]:
        return 1
df["pred_path1"]=df[["pred_pet_type","pred_pet_id"]].apply(ip,axis=1)

In [64]:
def ip(x):
    if x[0]=="dog":
        return -1
    if x[0]=="other":
        return -1
    if "," in x[0]:
        return -1
    if pd.isnull(x[1]):
        return -1
    if "jessie" in x[1]:
        return 0
    if "kitty" in x[1]:
        return 1
    if "lilo" in x[1]:
        return 2
    if "shadow" in x[1]: 
        return 3
    if "iris" in x[1]:
        return 4 
    if "new1" in x[1]:
        return 5
df["pred_path1"]=df[["pred_pet_type","pred_pet_id"]].apply(ip,axis=1)

In [78]:
from sklearn.metrics import classification_report,confusion_matrix 

print(classification_report(df["image_path1"],df["pred_path1"]))
print(confusion_matrix(df["image_path1"],df["pred_path1"]))



              precision    recall  f1-score   support

           0       1.00      0.82      0.90        22
           1       0.91      1.00      0.95        42

    accuracy                           0.94        64
   macro avg       0.96      0.91      0.93        64
weighted avg       0.94      0.94      0.94        64

[[18  4]
 [ 0 42]]


In [25]:
df.isnull().sum()

datetime          0
house_id          0
image_path        0
pet_type         27
pet_id           27
pred_pet_type     0
pred_pet_id       0
image_path1      12
pred_path1       12
dtype: int64